1) Реализовать простейший динамический массив, поддерживающий добавление элемента в конец массива при помощи метода append(e).
Также должны поддерживаться следующие операции:
•	извлечение элемента по индексу;
•	устанавка нового значения по индексу;
•	получение текущей длины массива.
1.1) Добавить реализацию удаления элемента из любого места массива и поддержку корректного вывода массива при помощи функции print.

In [1]:
class DynamicArray(object):
    def __init__(self, array = []):
        self.array = array #создаем массив
    def append(self, item): #Добавляет элемент в массив
        self.array.append(item)
    def get(self, index): #Получение элемента по индексу
        if 0 <= index < len(self.array):
            return self.array[index]
        else:
            raise IndexError("Index out of range")

    def set(self, index, value): #устанавка нового значения по индексу
        if 0 <= index < len(self.array):
            self.array[index] = value
        else:
            raise IndexError("Index out of range")

    def length(self): #получение текущей длины массива
        return len(self.array)

    def remove(self, item): #удаления элемента из любого места массива
        try:
            self.array.remove(item)
        except ValueError:
            pass

    def __str__(self): #поддержку корректного вывода массива при помощи функции print
        return str(self.array)
# Создаем объект динамического массива
arr = DynamicArray()

# Добавляем элементы в массив
arr.append(10)
arr.append(20)
arr.append(30)
print(arr)
print(arr.get(1))
arr.set(0, 150)
print(arr)
arr.remove(150)
print(arr.length())

[10, 20, 30]
20
[150, 20, 30]
2


2) Реализовать генератор, который возвращает значение поочередно извлекаемое из конца двух очередей (в качестве очереди используется deque из collections). Если очередь из которой извлекается элемент пуста - генератор заканчивает работу.

In [2]:
from collections import deque

def alternate_queue_values(queue1, queue2):
    while len(queue1) > 0 or len(queue2) > 0:
        if len(queue1) > 0:
            yield queue1.pop()
        if len(queue2) > 0:
            yield queue2.pop()

# Создаем две очереди
queue1 = deque([1, 2, 3, 4])
queue2 = deque(['a', 'b', 'c'])

# Применяем генератор для извлечения значений
gen = alternate_queue_values(queue1, queue2)
for value in gen:
    print(value)

4
c
3
b
2
a
1


3) Реализовать классы с медодом action():
а) Класс Pump - в методе action() извлекает очередное значение из генератора и помещает значение в очередь (очередь передается в конструктор).
b) Класс MultiAction - при вызове метода action() n раз вызвает метод action() класса, переданного в конструкторе. Число n также определяется в конструкторе.
c) Класс MultiPump - в методе action() извлекает очередное значение из генератора и помещает значение в одну из очередей (очереди передается в конструкторе); очереди , в которые помещаются очередные значения, меняются по порядку.


In [7]:
from collections import deque

class Pump:
    def __init__(self, queue):
        self.queue = queue
    
    def action(self, gen):
        try:
            value = next(gen)
            self.queue.append(value)
        except StopIteration:
            pass

class MultiAction:
    def __init__(self, obj, n):
        self.obj = obj
        self.n = n
    
    def action(self, gen):
        for _ in range(self.n):
            self.obj.action(gen)

class MultiPump:
    def __init__(self, queues):
        self.queues = queues
        self.queue_index = 0
    
    def action(self, gen):
        try:
            value = next(gen)
            self.queues[self.queue_index].append(value)
            self.queue_index = (self.queue_index + 1) % len(self.queues)
        except StopIteration:
            pass

# Создаем генератор
def number_generator():
    for i in range(1, 6):
        yield i

# Создаем очереди
queue1 = deque()
queue2 = deque()
queue3 = deque()

# Создаем экземпляры классов
pump1 = Pump(queue1)
pump2 = Pump(queue2)
multi_pump = MultiPump([queue1, queue2, queue3])

# Выполняем действия с генератором
gen = number_generator()
pump1.action(gen)
pump2.action(gen)

multi_act = MultiAction(pump1, 3)
multi_act.action(gen)

multi_pump.action(gen)
multi_pump.action(gen)
multi_pump.action(gen)
multi_pump.action(gen)

# Вывод содержимого очередей
print(queue1)
print(queue2)
print(queue3)


deque([1, 3, 4, 5])
deque([2])
deque([])


4) При помощи GenFromQ, Pump реализовать систему опработки сообщений. Сообщения создаются генератором сообщений возвращающим случайным образорм одно из сообщений. Сообщения из трех генераторов закачиваются в три очереди при помощи классов Pump, далее при помощи GenFromQ и Pump объединяются в одну очередь и выводятся на экран (можно реализовать при помощи класса с action и вызываемого при помощи MultiPump).



In [16]:
import random
from collections import deque

class GenFromQ:
    def __init__(self, queues):
        self.queues = queues
    
    def action(self, output_queue):
        for queue in self.queues:
            while queue:
                output_queue.append(queue.popleft())

class Pump:
    def __init__(self, queue):
        self.queue = queue
    
    def action(self, gen):
        try:
            value = next(gen)
            self.queue.append(value)
        except StopIteration:
            pass

# Генератор случайных сообщений
def message_generator():
    messages = ["Important message", "Urgent message", "Normal message"]
    while True:
        yield random.choice(messages)

# Создаем три очереди
queue1 = deque()
queue2 = deque()
queue3 = deque()
output_queue = deque()

# Создаем экземпляры классов Pump для каждой очереди
pump1 = Pump(queue1)
pump2 = Pump(queue2)
pump3 = Pump(queue3)

# Создаем экземпляр класса GenFromQ для объединения сообщений из разных очередей
gen_from_q = GenFromQ([queue1, queue2, queue3])

# Создаем генератор сообщений
gen = message_generator()

# Загружаем сообщения в очереди
pump1.action(gen)
pump2.action(gen)
pump3.action(gen)

# Объединяем сообщения из разных очередей в одну
gen_from_q.action(output_queue)

# Выводим сообщения на экран
while output_queue:
    print(output_queue.popleft())


Urgent message
Important message
Normal message
